<a href="https://colab.research.google.com/github/sharsulkar/H1B_LCA_outcome_prediction/blob/main/prototyping/notebooks/05_sh_vizualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pickle import dump, load
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
#Custom transformer to drop rows based on filter
class droprows_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      self.row_index = None # row index to drop
      self.inplace=True
      self.reset_index=True

    def fit( self, X, y=None):
      return self 
    
    def transform(self, X, y=None):
      self.row_index=X[~X.CASE_STATUS.isin(['Certified','Denied'])].index
      X.drop(index=self.row_index,inplace=self.inplace)
      if self.reset_index:
        X.reset_index(inplace=True,drop=True)
      return X

In [ ]:
class buildfeatures_Transformer(BaseEstimator, TransformerMixin):
  def __init__(self, input_columns):
    self.input_columns=input_columns

  def date_diff(self,date1,date2):
    return date1-date2

  def is_USA(self,country):
    if country=='UNITED STATES OF AMERICA':
      USA_YN='Y' 
    else:
      USA_YN='N'
    return USA_YN

  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    # Processing_Days and Validity_days
    X['PROCESSING_DAYS']=self.date_diff(X.DECISION_DATE, X.RECEIVED_DATE).dt.days
    X['VALIDITY_DAYS']=self.date_diff(X.END_DATE, X.BEGIN_DATE).dt.days

    # SOC_Codes
    X['SOC_CD2']=X.SOC_CODE.str.split(pat='-',n=1,expand=True)[0]
    X['SOC_CD4']=X.SOC_CODE.str.split(pat='-',n=1,expand=True)[1].str.split(pat='.',n=1,expand=True)[0]
    X['SOC_CD_ONET']=X.SOC_CODE.str.split(pat='-',n=1,expand=True)[1].str.split(pat='.',n=1,expand=True)[1]

    # USA_YN
    X['USA_YN']=X.EMPLOYER_COUNTRY.apply(self.is_USA)

    # Employer_Worksite_YN
    X['EMPLOYER_WORKSITE_YN']='Y'
    X.loc[X.EMPLOYER_POSTAL_CODE.ne(X.WORKSITE_POSTAL_CODE),'EMPLOYER_WORKSITE_YN']='N'

    # OES_YN
    X['OES_YN']='Y'
    X.iloc[X[~X.PW_OTHER_SOURCE.isna()].index,X.columns.get_loc('OES_YN')]='N'

    # SURVEY_YEAR
    X['SURVEY_YEAR']=pd.to_datetime(X.PW_OES_YEAR.str.split(pat='-',n=1,expand=True)[0]).dt.to_period('Y')
    PW_other_year=X[X.OES_YN=='N'].PW_OTHER_YEAR
    #Rename the series and update dataframe with series object
    PW_other_year.rename("SURVEY_YEAR",inplace=True)
    X.update(PW_other_year)

    # WAGE_ABOVE_PREVAILING_HR
    X['WAGE_PER_HR']=X.WAGE_RATE_OF_PAY_FROM
    #compute for Year
    X.iloc[X[X.WAGE_UNIT_OF_PAY=='Year'].index,X.columns.get_loc('WAGE_PER_HR')]=X[X.WAGE_UNIT_OF_PAY=='Year'].WAGE_RATE_OF_PAY_FROM/2067
    #compute for Month
    X.iloc[X[X.WAGE_UNIT_OF_PAY=='Month'].index,X.columns.get_loc('WAGE_PER_HR')]=X[X.WAGE_UNIT_OF_PAY=='Month'].WAGE_RATE_OF_PAY_FROM/172

    #initialize with WAGE_RATE_OF_PAY_FROM
    X['PW_WAGE_PER_HR']=X.PREVAILING_WAGE
    #compute for Year
    X.iloc[X[X.PW_UNIT_OF_PAY=='Year'].index,X.columns.get_loc('PW_WAGE_PER_HR')]=X[X.PW_UNIT_OF_PAY=='Year'].PREVAILING_WAGE/2067
    #compute for Month
    X.iloc[X[X.PW_UNIT_OF_PAY=='Month'].index,X.columns.get_loc('PW_WAGE_PER_HR')]=X[X.PW_UNIT_OF_PAY=='Month'].PREVAILING_WAGE/172

    X['WAGE_ABOVE_PW_HR']=X.WAGE_PER_HR-X.PW_WAGE_PER_HR

    return X

In [ ]:
#Custom transformer to drop features for input feature list
class dropfeatures_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns, inplace):
      self.columns = columns # list of categorical columns in input Dataframe
      self.inplace=True

    def fit( self, X, y=None):
      return self 
    
    def transform(self, X, y=None):
      X.drop(columns=self.columns,inplace=self.inplace)
      return X

In [ ]:
#Custom transformer to compute Random Standard encoding for categorical features for incrementaly encoding data
class RSE_Transformer(BaseEstimator, TransformerMixin):
    #Class Constructor
    def __init__( self, cat_cols, categories=None, RSE=None ):
        self.cat_cols = cat_cols # list of categorical columns in input Dataframe
        self.categories = None # Array of unique non-numeric values in each categorical column
        self.RSE = None # Array of Random Standard encoding for each row in categories
        
    def fit( self, X, y=None ):
      #Get a list of all unique categorical values for each column
      if self.categories is None:
        self.categories = [X[column].unique() for column in cat_cols]

        #replace missing values and append missing value label to each column to handle missing values in test dataset that might not be empty in train dataset
        for i in range(len(self.categories)):
          if np.array(self.categories[i].astype(str)!=str(np.nan)).all():
            self.categories[i]=np.append(self.categories[i],np.nan)

        #compute RandomStandardEncoding 
        self.RSE=[np.random.normal(0,1,len(self.categories[i])) for i in range(len(self.cat_cols))]

      else:
        for i in range(len(self.cat_cols)):
          #append new unique categories to self.categories
          new_categories=list(set(X[self.cat_cols[i]].unique()).difference(set(self.categories[i])))
          if new_categories!=[]:
            #print('not empty') #replace with logging call
            #print('categories before append',len(categories[i])) #logging call
            self.categories[i]=np.append(self.categories[i],new_categories) #append new categories to the end
            new_RSE=np.random.normal(0,1,len(new_categories)) #generate new RSE values
            #regenrate if overlap found with existing encodings
            if set(new_RSE).issubset(set(self.RSE[i])): 
              #print('yes') #loggin call
              new_RSE=np.random.normal(0,1,len(new_categories))
            
            self.RSE[i]=np.append(self.RSE[i],new_RSE) #append new RSE values
          #print('new categories',len(new_categories)) #logging call
          #print('categories after append',len(categories[i]))
     
      return self 
    
    def transform(self, X, y=None):
      for i in range(len(self.cat_cols)):
        #Temporary measure to handle previously unseen values
        #replace unseen values with NaN
        X.loc[X[~X[(str(self.cat_cols[i]))].isin(self.categories[i])].index,(str(self.cat_cols[i]))]=np.NaN

        #replace seen values with encoding
        X.loc[:,(str(self.cat_cols[i]))].replace(dict(zip(self.categories[i], self.RSE[i])),inplace=True)
      return X    

    def inverse_transform(self,X):
      for i in range(len(self.cat_cols)):
        X.loc[:,(str(self.cat_cols[i]))].replace(dict(zip(self.RSE[i], self.categories[i])),inplace=True)
      return X

In [ ]:
#custom transformer for incrementally scaling to standard scale using pooled mean and variance
class CustomStandardScaler(BaseEstimator, TransformerMixin):
  def __init__(self,mean=None,var=None,n_samples_seen=None,scale=None):
    self.mean=None #mean
    self.var=None
    self.n_samples_seen=None
    self.scale=None

  def compute_sample_mean(self,X):
    return np.mean(X,axis=0)

  def compute_sample_var(self,X):
    return np.var(X,axis=0)

  def compute_sample_size(self,X):
    #assuming X is imputed, if there are null values, throw error aksing that X be imputed first
    return len(X)

  def compute_pooled_mean(self,X):
    #compute the sample mean and size
    sample_mean=self.compute_sample_mean(X)
    sample_count=self.compute_sample_size(X) 
    #compute pool mean
    pool_mean=(self.mean*self.n_samples_seen + sample_mean*sample_count)/(self.n_samples_seen + sample_count)

    return pool_mean

  def compute_pooled_var(self,X):
    #compute the sample var and size
    sample_var=self.compute_sample_var(X)
    sample_count=self.compute_sample_size(X) 
    #compute pool variance
    pool_var=(self.var*(self.n_samples_seen - 1) + sample_var*(sample_count - 1))/(self.n_samples_seen + sample_count - 2)

    return pool_var

  def fit(self,X):
    if self.mean is None:
      self.mean=self.compute_sample_mean(X)
    else: 
      self.mean=self.compute_pooled_mean(X)
    
    if self.var is None:
      self.var=self.compute_sample_var(X)
    else: 
      self.var=self.compute_pooled_var(X)

    if self.n_samples_seen is None:
      self.n_samples_seen=self.compute_sample_size(X) 
    else: 
      self.n_samples_seen+=self.compute_sample_size(X)
    return self

  def transform(self,X):
    return (X-self.mean)/np.sqrt(self.var)

  def inverse_transform(self,X):
    return X*np.sqrt(self.var) + self.mean



In [ ]:
def read_csv_to_list(filepath,header=None,squeeze=True):
  return list(pd.read_csv(filepath,header=None,squeeze=True))

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Datasets/LCA_dataset_sample1000.xlsx')

In [ ]:
model=load(open('/content/drive/MyDrive/saved_models/H1B_LCA_prediction/adaboost_batch_train.pkl','rb'))
build_feature_pipe=load(open('/content/drive/MyDrive/saved_models/H1B_LCA_prediction/build_feature_pipe_batch_train.pkl','rb'))
preprocess_pipe=load(open('/content/drive/MyDrive/saved_models/H1B_LCA_prediction/pipeline_batch_train.pkl','rb'))

In [ ]:
fe_df=build_feature_pipe.transform(df)

In [ ]:
#Display outcome statistics for -
#Specific to the test sample -
#employer name - stats for current employer
#NAICS code - current Naics code
#SOC_Code - current Soc code

#Generic stats from training data-
#visa class, Wage level, AGREE_TO_LC_STATEMENT, H-1B_DEPENDENT, WILLFUL_VIOLATOR, PUBLIC_DISCLOSURE, EMPLOYER_WORKSITE_YN
#validity days=3y vs <3y 

In [ ]:
def generate_feature_stats(df,column,stats_file_path=None):
  if stats_file_path is None:
    #create new file
    stats_df=pd.DataFrame(index=np.unique(df[column]),columns=['Certified','Denied'])
    for index in stats_df.index:
      stats_df.loc[index,['Certified']]=df[(df[column]==index) & (df.CASE_STATUS=='Certified')].shape[0]
      stats_df.loc[index,['Denied']]=df[(df[column]==index) & (df.CASE_STATUS=='Denied')].shape[0]

  else:
    #import the existing file and append results
    stat_df=pd.read_csv(stats_file_path,index=0)
    new_index=np.unique(df[column])
    for index in new_index:
      if index in stat_df.index: #rewrite with correct condition
        stats_df.loc[index,['Certified']]+=df[(df[column]==index) & (df.CASE_STATUS=='Certified')].shape[0]
        stats_df.loc[index,['Denied']]+=df[(df[column]==index) & (df.CASE_STATUS=='Denied')].shape[0]
      else:
        stats_df.loc[index,['Certified']]=df[(df[column]==index) & (df.CASE_STATUS=='Certified')].shape[0]
        stats_df.loc[index,['Denied']]=df[(df[column]==index) & (df.CASE_STATUS=='Denied')].shape[0]
  
  #save the df as csv
  return stats_df

In [ ]:
def compute_stats_positive_class(column,value,stats_file_path=None):
  #import the file into a df
  stat_df=pd.read_csv(stats_file_path,index=0)
  #find the index
  stats_arr=np.array(stats_df.loc[index])
  #return the 
  return stats_arr[0]*100/stats_arr.sum()

In [ ]:
#if prediction is denied, create a grid of all possible values of variable features, run them through the model and display those that return a positive outcome as suggestions 
#constant feature list - VISA_CLASS, EMPLOYER_NAME, EMPLOYER_POSTAL_CODE, EMPLOYER_COUNTRY, NAICS_CODE, SECONDARY_ENTITY, PREVAILING_WAGE, SOC_CODE, SOC_TITLE, SURVEY RELATED COLUMNS, 
#variable features list - FULL_TIME_POSITION, PW_WAGE_LEVEL,  AGREE_TO_LC_STATEMENT, H-1B_DEPENDENT, WILLFUL_VIOLATOR, PUBLIC_DISCLOSURE, NEW_EMPLOYMENT, CONTINUED_EMPLOYMENT, CHANGE_PREVIOUS_EMPLOYMENT, NEW_CONCURRENT_EMPLOYMENT, CHANGE_EMPLOYER,
#AMENDED_PETITION, AGENT_REPRESENTING_EMPLOYER, EMPLOYER_WORKSITE_YN, WAGE_ABOVE_PW_HR

In [ ]:
def return_allcombinations(arr):
  #calculate dimensions of final array
  cum_prod=np.cumprod([len(arr[i]) for i in range(len(arr))])
  m=np.prod([len(arr[i]) for i in range(len(arr))])
  hlayer_arr=np.array(arr[0]).repeat(m/cum_prod[0])
  for i in range(1,len(arr)):
      cc=np.array(arr[i]).repeat(m/cum_prod[i])

      for j in range(np.int(cum_prod[i-1])-1):
        cc=np.hstack((cc,np.array(arr[i]).repeat(m/cum_prod[i])))
      hlayer_arr=np.vstack((hlayer_arr,cc))
  return hlayer_arr.T

In [ ]:
denied_df=fe_df[fe_df.CASE_STATUS=='Denied'].reset_index(drop=True)
denied_df.pop('CASE_STATUS')
#user input record that needs to be predicted
X_pred=pd.DataFrame(denied_df.iloc[7]).T
X_arr=preprocess_pipe.transform(X_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [ ]:
#define variable column list that will be used for grid search
#create array of unique values for all above columns sourcing from the RSE transform
#arr=[['Y','N'],['I','II','III','IV'],['Y','N'],['Y','N'],['Y','N'],['Disclose Business', 'Disclose Employment', 'Disclose Business and Employment'],['Y','N'],['Y','N'],['Y','N'],[0,10],[100,500,1095]]
arr=[['Y','N'],['I','II','III','IV'],['Y','N'],['Y','N'],['Y','N'],['Disclose Business', 'Disclose Employment','Disclose Business and Employment'],['Y','N'],['Y','N'],['Y','N']]
#append wage information
wage_arr=[0,10]
if X_pred.WAGE_ABOVE_PW_HR.values[0]>0:
  wage_arr=np.append(wage_arr,X_pred.WAGE_ABOVE_PW_HR.values[0])
arr.append(list(wage_arr))
#append validity days information
validity_days_arr=[100,1095]
if X_pred.VALIDITY_DAYS.values[0]<1095:
  validity_days_arr=np.append(validity_days_arr,X_pred.VALIDITY_DAYS.values[0])
arr.append(list(validity_days_arr))

#create grid array for all possible combinations
grid_arr=return_allcombinations(arr)
grid_len=grid_arr.shape[0]

#generate base grid dataframe by repeating the X_pred sample grid_len times
X_reconstructed=X_pred.iloc[np.arange(1).repeat(grid_len)].reset_index(drop=True)

#update the grid dataframe with the grid array
var_columns=['FULL_TIME_POSITION', 'PW_WAGE_LEVEL',  'AGREE_TO_LC_STATEMENT', 'H-1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'PUBLIC_DISCLOSURE', 'AGENT_REPRESENTING_EMPLOYER','EMPLOYER_WORKSITE_YN', 'OES_YN','WAGE_ABOVE_PW_HR','VALIDITY_DAYS']
pd.DataFrame.update(X_reconstructed,pd.DataFrame(grid_arr,columns=var_columns))

#apply pipeline to the recon
X_reconstructed_arr=preprocess_pipe.transform(X_reconstructed)

y=model.predict(X_reconstructed_arr)
len(y[y==0])

/usr/local/lib/python3.6/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


13696

In [ ]:
cat_cols=read_csv_to_list('https://raw.githubusercontent.com/sharsulkar/H1B_LCA_outcome_prediction/main/data/processed/categorical_columns.csv',header=None,squeeze=True)
num_cols=read_csv_to_list('https://raw.githubusercontent.com/sharsulkar/H1B_LCA_outcome_prediction/main/data/processed/numeric_columns.csv',header=None,squeeze=True)
print(np.append(cat_cols,num_cols))
'''
#column indices of features of intrest after dropfeatures_Transformer
['FULL_TIME_POSITION' - 3,  
 'AGENT_REPRESENTING_EMPLOYER' - 5, 
 'PW_WAGE_LEVEL' - 7,
 'AGREE_TO_LC_STATEMENT'-8, 
 'H-1B_DEPENDENT'-9,
 'WILLFUL_VIOLATOR'-10,
 'PUBLIC_DISCLOSURE'-11,
 'EMPLOYER_WORKSITE_YN'-16, 
 'OES_YN' - 17,
 'VALIDITY_DAYS'-29,
 'WAGE_ABOVE_PW_HR' - 30]
 '''

['VISA_CLASS' 'SOC_TITLE' 'EMPLOYER_NAME' 'FULL_TIME_POSITION'
 'NAICS_CODE' 'AGENT_REPRESENTING_EMPLOYER' 'SECONDARY_ENTITY'
 'PW_WAGE_LEVEL' 'AGREE_TO_LC_STATEMENT' 'H-1B_DEPENDENT'
 'WILLFUL_VIOLATOR' 'PUBLIC_DISCLOSURE' 'SOC_CD2' 'SOC_CD4' 'SOC_CD_ONET'
 'USA_YN' 'EMPLOYER_WORKSITE_YN' 'OES_YN' 'SURVEY_YEAR'
 'TOTAL_WORKER_POSITIONS' 'NEW_EMPLOYMENT' 'CONTINUED_EMPLOYMENT'
 'CHANGE_PREVIOUS_EMPLOYMENT' 'NEW_CONCURRENT_EMPLOYMENT'
 'CHANGE_EMPLOYER' 'AMENDED_PETITION' 'WORKSITE_WORKERS'
 'TOTAL_WORKSITE_LOCATIONS' 'PROCESSING_DAYS' 'VALIDITY_DAYS'
 'WAGE_ABOVE_PW_HR']


"\n#column indices of features of intrest after dropfeatures_Transformer\n['FULL_TIME_POSITION' - 3,  \n 'AGENT_REPRESENTING_EMPLOYER' - 5, \n 'PW_WAGE_LEVEL' - 7,\n 'AGREE_TO_LC_STATEMENT'-8, \n 'H-1B_DEPENDENT'-9,\n 'WILLFUL_VIOLATOR'-10,\n 'PUBLIC_DISCLOSURE'-11,\n 'EMPLOYER_WORKSITE_YN'-16, \n 'OES_YN' - 17,\n 'VALIDITY_DAYS'-29,\n 'WAGE_ABOVE_PW_HR' - 30]\n "

In [ ]:
def weightedL2(a, b, w):
    q = a-b
    return np.sqrt((w*q*q).sum())

In [ ]:
#set weights to one
w=np.ones(len(X_arr[0])) #initialize weights with 1

#change weights of some fields to give them priority over others
variable_idx=[3,5,7,8,9,10,11,16,17,29,30] # index of variable fields where changes in values can impact outcome
w[variable_idx]=[1,1,2,1,1,1,1,1,1,2,2] #increase weights of PW_WAGE_LEVEL,VALIDITY_DAYS,WAGE_ABOVE_PW_HR as those as most easy to change for the user

In [ ]:
if len(y[y==0])>0: #check if any of the combination will result in a confirmation

  #Calculate weighted eucledian distance between the X_pred and each row in grid and return index of minimum distance
  distance_arr=[]
  for i in range(grid_len):
    if y[i]==0:
      #q=X_arr[0]-X_reconstructed_arr[i]
      #L2_distance=np.sqrt((w*q*q).sum())
      L2_distance=weightedL2(X_arr[0], X_reconstructed_arr[i], w)
      if L2_distance==0: #distance between X_pred and itself in the grid is 0
        L2_distance+=1 #add offset so it does not return the same record if it is misclassified to class 0
      distance_arr=np.append(distance_arr,L2_distance) #append the distance

  #find the first suggesting with minimum changes to the application
  min_change_index=np.argmin(distance_arr)
  #np.argsort(distance_arr)[:3] #return indices for min 3 records

  print('The model predicts that application\'s outcome can change from Denied to Confirmed if all the below changes are made:')
  #print the suggested changes that will result in a approved application
  for index in np.nonzero(X_arr[0] - X_reconstructed_arr[min_change_index])[0]:
    #print(index)
    
    feature_name=np.append(cat_cols,num_cols)[index]
    current_value=X_pred[feature_name].iloc[0]
    new_value=X_reconstructed[feature_name].iloc[min_change_index]
    #replace nan with missing
    if current_value==np.NaN:
       current_value='missing'
    #handle specific features
    if (feature_name=='VALIDITY_DAYS' and current_value>=1094):
      #dont suggest reducing validity days, more the better
      continue

    elif feature_name=='WAGE_ABOVE_PW_HR':
      if current_value>float(new_value):
      #dont suggest reducing the wage
        continue
      else:
        print('Consider Increasing the per hour wage rate by',new_value,' over the given per hour prewailing wage.')

    elif feature_name=='EMPLOYER_WORKSITE_YN':
      if current_value=='Y':
      #dont suggest reducing the wage
        print('Consider changing the worksite location to a postal code other than the employer address.')
      else:
        print('Consider changing the worksite location to a postal code same as the employer address.')

    elif feature_name=='OES_YN':
      #OES_YN is a derived feature, give explaination that end user can understand
      if current_value=='Y':
        print('Consider using survey other than OES.')
      else:
        print('Consider using OES survey.')
    
    else:
      print('Consider changing ',feature_name, ' from ',current_value,' to ',new_value)
else:
  print('We are sorry. The model does not predict a change in your application outcome with current parameters. Please check the documentation for the complete list of parameters used.')
  print('Please consider consulting with a qualified immigration professional regarding your application.')

The model predicts that application's outcome can change from Denied to Confirmed if all the below changes are made:
Consider changing  PW_WAGE_LEVEL  from  nan  to  I


In [ ]:
denied_df[var_columns].iloc[7]

FULL_TIME_POSITION                             Y
PW_WAGE_LEVEL                                NaN
AGREE_TO_LC_STATEMENT                          Y
H-1B_DEPENDENT                                 N
WILLFUL_VIOLATOR                               N
PUBLIC_DISCLOSURE              Disclose Business
AGENT_REPRESENTING_EMPLOYER                    Y
EMPLOYER_WORKSITE_YN                           Y
OES_YN                                         N
WAGE_ABOVE_PW_HR                         9.00372
VALIDITY_DAYS                                376
Name: 7, dtype: object

In [ ]:
X_reconstructed[var_columns].iloc[665]

FULL_TIME_POSITION                             Y
PW_WAGE_LEVEL                                  I
AGREE_TO_LC_STATEMENT                          Y
H-1B_DEPENDENT                                 N
WILLFUL_VIOLATOR                               N
PUBLIC_DISCLOSURE              Disclose Business
AGENT_REPRESENTING_EMPLOYER                    Y
EMPLOYER_WORKSITE_YN                           Y
OES_YN                                         N
WAGE_ABOVE_PW_HR               9.003715529753258
VALIDITY_DAYS                                376
Name: 665, dtype: object

Decision - According to the prediction model trained on historical data, your application is expected to be [Decision - Approved/Denied].  
Observations and Statistics -   
1. If [employer_name] is found in RSE.categories - 
[employer_approval_percent]% of application by your employer [employer_name] get approved.  
else ignore this observation.
2. [naics_approval_percent]% of application in your NAICS code [naics_code] get approved.  
3. [soc_approval_percent]% of application in your SOC code [soc_code] get approved.  
if validity_days<3 years -  
4. Your LCA application validity date is less than the maximum allowed period of 3 years.  

Recommendations -  
if Decision = Approved - Congratulations! No more recommendations.  
if decision = Denied - Do grid serch on variable feature list  
if model gives positive outcome for grid search -  
The model predicts a positive outcome if below changes are made to the applications -  
Show top 3 set of changes.  
if model gives negative outcome for grid search -  
Sorry, the model could not predict a positive outcome for your application based on historical data available. Contact an immigration professional for further advice.

